# Create Project

In [2]:
#import libraries 
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Stats materials
from scipy import stats
import scipy.stats as stats

# import SQL materials
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists
import pymysql
pymysql.install_as_MySQLdb()
from urllib.parse import quote_plus as urlquote
from sqlalchemy_utils import create_database, database_exists

In [4]:
# import the data
filename = 'Data4_March31/tmdb_results_combined.csv.gz'
firsthypo_df=pd.read_csv(filename)
#check that the data loaded
firsthypo_df.head(2)

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.5,22.0,NaN


In [5]:
#check info
firsthypo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2544 entries, 0 to 2543
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                2544 non-null   object 
 1   adult                  2542 non-null   float64
 2   backdrop_path          1369 non-null   object 
 3   belongs_to_collection  204 non-null    object 
 4   budget                 2542 non-null   float64
 5   genres                 2542 non-null   object 
 6   homepage               171 non-null    object 
 7   id                     2542 non-null   float64
 8   original_language      2542 non-null   object 
 9   original_title         2542 non-null   object 
 10  overview               2492 non-null   object 
 11  popularity             2542 non-null   float64
 12  poster_path            2284 non-null   object 
 13  production_companies   2542 non-null   object 
 14  production_countries   2542 non-null   object 
 15  rele

# Preparing Data

In [6]:
# check the value counts on certification column for first hypothesis
firsthypo_df['certification'].value_counts()


R          458
PG-13      183
NR          72
PG          62
G           24
NC-17        6
Unrated      1
-            1
Name: certification, dtype: int64

In [7]:
# explore the  missing data
num_missing = firsthypo_df['certification'].isna().sum()

total_rows = firsthypo_df.shape[0]

percent_missing = num_missing / total_rows
print(f'{percent_missing:.2f}% of the data in the certification column is missing')

0.68% of the data in the certification column is missing


In [8]:
# find the row with - rating
firsthypo_df.loc[firsthypo_df['certification'] == '-']

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
2056,tt0282526,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,172550.0,it,Giorni,...,0.0,90.0,"[{'english_name': 'Italian', 'iso_639_1': 'it'...",Released,NaN,Days,0.0,5.4,8.0,-


In [11]:
#  drop the row, it has 0 revenue
#firsthypo_df=firsthypo_df.dropna(firsthypo_df['certification']==['-'], inplace=True)
#firsthypo_df.drop(2056)
firsthypo_df = firsthypo_df.loc[firsthypo_df['certification'] != '-']

In [17]:
# check the value counts on certification column for first hypothesis
firsthypo_df['certification'].value_counts()


R          458
PG-13      183
NR          72
PG          62
G           24
NC-17        6
Unrated      1
Name: certification, dtype: int64

# First Hypothesis

## Hypothesis
- Null: Movies have the same revenue in all MPAA ratings
- Alternate: Movies with different MPAA ratings  have different revenue.

- Possible question would me how much? See Project 5 for linear regression

## Test type: ANOVA

In [12]:
## Create groups dictionary. 
groups = {}
## Loop through all unique categories
for i in firsthypo_df['certification'].unique():
    ## Get series for group and rename
    data = firsthypo_df.loc[firsthypo_df['certification']==i,'revenue'].copy()
    
    # save into the dictionary
    groups[i] = data
groups.keys()

dict_keys([nan, 'PG', 'R', 'G', 'NR', 'PG-13', 'NC-17', 'Unrated'])

## Check Assumptions for ANOVA
- outliers
- normality
- equal variance

In [16]:
## Running normal test on each group and confirming there are >20 in each group
norm_results = {}
for i, data in groups.items():
    stat, p = stats.normaltest(data)
    ## save the p val, test statistic, and the size of the group
    norm_results[i] = {'n': len(data),
                             'p':p,
                             'test stat':p<0.05}
## convert to a dataframe
norm_results_df = pd.DataFrame(norm_results).T
norm_results_df

ValueError: skewtest is not valid with less than 8 samples; 0 samples were given.

In [21]:
# check for equal variance
stats.levene(*groups.values())

LeveneResult(statistic=nan, pvalue=nan)